# Ch.08 ScrapyとHTTP

<https://sugiaki1989.gitbook.io/scrapy-note/chapter08_http>

## HTTPについて

- HTTP
  - WebブラウザがWebサーバーと通信するためのプロトコル
  - TCP/IPにおけるアプリケーションがやり取りを行う層のプロトコル
  - トランスポート層の中に位置する
  - HTMLを含めさまざまな情報を取得するために通信方法
    - Webブラウザがプロトコルに従って、Webサーバーにリクエストを送る
    - Webサーバーは、そのリクエストを受け取って、Webブラウザにレスポンスを返す
- Web
  - World Wide Webを略したもの
- Webページ
  - HTML(HyperText Markup language)という言語で構成されている

## HTTPの内容

```bash
# curlコマンド: ファイルを送受信するためのコマンド
curl --version
curl 7.88.1 (x86_64-apple-darwin22.0) libcurl/7.88.1 (SecureTransport) LibreSSL/3.3.6 zlib/1.2.11 nghttp2/1.51.0
Release-Date: 2023-02-20
Protocols: dict file ftp ftps gopher gophers http https imap imaps ldap ldaps mqtt pop3 pop3s rtsp smb smbs smtp smtps telnet tftp
Features: alt-svc AsynchDNS GSS-API HSTS HTTP2 HTTPS-proxy IPv6 Kerberos Largefile libz MultiSSL NTLM NTLM_WB SPNEGO SSL threadsafe UnixSocket
```

### リクエストを送る

```bash
curl --version http://example.com
```

```bash
*   Trying 93.184.216.34:80...
* Connected to example.com (93.184.216.34) port 80 (#0)
> GET / HTTP/1.1
> Host: example.com
> User-Agent: curl/7.88.1
> Accept: */*
> 
< HTTP/1.1 200 OK
< Accept-Ranges: bytes
< Age: 205325
< Cache-Control: max-age=604800
< Content-Type: text/html; charset=UTF-8
< Date: Sun, 02 Jul 2023 10:40:48 GMT
< Etag: "3147526947+gzip"
< Expires: Sun, 09 Jul 2023 10:40:48 GMT
< Last-Modified: Thu, 17 Oct 2019 07:18:26 GMT
< Server: ECS (sab/5693)
< Vary: Accept-Encoding
< X-Cache: HIT
< Content-Length: 1256
< 
<!doctype html>
<html>
<head>
    <title>Example Domain</title>

    <meta charset="utf-8" />
    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <style type="text/css">
    body {
        background-color: #f0f0f2;
        margin: 0;
        padding: 0;
        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;
        
    }
    div {
        width: 600px;
        margin: 5em auto;
        padding: 2em;
        background-color: #fdfdff;
        border-radius: 0.5em;
        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);
    }
    a:link, a:visited {
        color: #38488f;
        text-decoration: none;
    }
    @media (max-width: 700px) {
        div {
            margin: 0 auto;
            width: auto;
        }
    }
    </style>    
</head>

<body>
<div>
    <h1>Example Domain</h1>
    <p>This domain is for use in illustrative examples in documents. You may use this
    domain in literature without prior coordination or asking for permission.</p>
    <p><a href="https://www.iana.org/domains/example">More information...</a></p>
</div>
</body>
</html>
* Connection #0 to host example.com left intact
```

## HTTPリクエスト

```bash
# リクエスト部分
*   Trying 93.184.216.34:80...
* Connected to example.com (93.184.216.34) port 80 (#0)
> GET / HTTP/1.1           # リクエスト業
> Host: example.com        # HTTPヘッダー
> User-Agent: curl/7.88.1  # HTTPヘッダー
> Accept: */*              # HTTPヘッダー
>                          # HTTPヘッダーの終わり
```

|メソッド|内容|
|:-|:-|
|GET|指定したURLのリソースをリクエスト|
|HEAD|GETリクエストと同じレスポンスを求めるが、レスポンス本文はなく、ヘッダーのみ|
|POST|指定したリソースに情報を送信するためのメソッド|
|PUT|指定したURLにリソースを保存|
|DELETE|指定したURLのリソースを削除|
|OPTIONS|指定したURLの通信オプションを示すために使用|
|TRACE|サーバーまでのネットワーク経路をチェック|
|PATCH|リソースを部分的に変更|
|CONNECT|TCPトンネルを接続する。暗号化したメッセージをプロキシサーバーを経由して転送する際に使用|

&nbsp;

- RESTアーキテクチャ
  - POST(Create), GET(Read), PUT(Update), DELETE(Delete)の操作をまとめてCRUD操作として考えること

## HTTPレスポンス

```bash
# レスポンス部分
< HTTP/1.1 200 OK                                 # ステータス行 スタータスコード: 200(成功)
< Accept-Ranges: bytes                            # HTTPヘッダー
< Age: 205325
< Cache-Control: max-age=604800
< Content-Type: text/html; charset=UTF-8          # キャラセット: UTF-8
< Date: Sun, 02 Jul 2023 10:40:48 GMT
< Etag: "3147526947+gzip"
< Expires: Sun, 09 Jul 2023 10:40:48 GMT
< Last-Modified: Thu, 17 Oct 2019 07:18:26 GMT
< Server: ECS (sab/5693)
< Vary: Accept-Encoding
< X-Cache: HIT                                   # キャッシュ
< Content-Length: 1256                           # コンテンツの長さ
<                                                # ヘッダー行の終わり
<!doctype html>                                  # メッセージボディ
<html>
<head>
    # 略
</head>

<body>
    # 略
</body>
</html>
* Connection #0 to host example.com left intact
```

- 同期通信
  - HTTPリクエストとHTTPレスポンスを交互に通信して、Webページのコンテンツを表示する
  - ページの更新に時間がかかる
- Ajax(Asynchronous JavaScript + XML)
  - Webブラウザ上で動作するJavaScriptが直接Webサーバーと通信を行い、表示するHTMLを更新する仕組み
  - 非同期通信

## HTTPステータスコード

|コード|メッセージ|内容|
|:-|:-|:-|
|100|Continue|リクエストを継続している|
|200|OK|リクエストが正常に完了している|
|301|Moved Permanently|リクエストされたコンテンツが移動している|
|302|Found|リクエストされたコンテンツが一時的に移動している|
|304|Not Modified|リクエストされたコンテンツが未更新。Webブラウザに一時的に保存されたコンテンツが表示されている。|
|400|Bad Request|リクエストが不正である。|
|401|Unauthorized|アクセス認証が必要。|
|403|Forbidden|アクセス禁止。|
|404|Not Found|リクエストされたコンテンツがない。|
|405|Method Not Allowed|そのメソッドでのリクエストが許可されていない。|
|406|Not Acceptable|ヘッダーで指定した言語でレスポンスできない。|
|408|Request Timeout|クライアントとサーバー間の通信時間が設定時間をオーバーした。|
|500|Internal Server Error|リクエスト中にサーバーでエラーが発生している。|
|501|Not Implemented|そのメソッドがサーバーでサポートされていない。|
|502|Bad Gateway|ゲートウェイやプロキシとして動作しているサーバーが上位のサーバーから不正なレスポンスを受け取った。|
|503|Service Unavailable|リクエストしたサーバーが一時的に停止している。|

## HTTPヘッダー

- HTTPヘッダー
  - 一般ヘッダー
    - Connection: リクエスト後のTCPコネクションの接続状態に関する通知
    - Date: HTTPメッセージが作成された日時
    - Update: HTTPのバージョンをアップデートするように通知
    - Cache-Control: キャッシュの動作を指定
    - Pragma: データのキャッシュなどの追加情報
    - Transfer-Encoding: ボディで送るデータのエンコード方式
  - リクエストヘッダー
    - Accept: クライアントの受け入れ可能なコンテンツ
    - Accept-Charset: クライアントの受け入れ可能な文字セット
    - Accept-Encoding: クライアントの受け入れ可能な文字エンコード
    - Accept-Language: クライアントの受け入れ可能な言語
    - Cookie: Cookieをサーバーに送信する
    - From: リクエスト者のメールアドレス
    - Host: リクエスト先のサーバー
    - Referer: 直前にリンクしていたURL
    - User-Agent: Webブラウザの情報
    - Proxy-Authorization: プロキシに対する認証情報
  - レスポンスヘッダー
    - Age: ボディで送信するデータの経過秒数
    - Allow: URLに対して使用可能なメソッド
    - Proxy-Authenticate: プロキシでの認証が必要なこと
    - Retry-After: 次のリクエストを送るまでの待機時間
    - Location: リダイレクト先のWebページの情報
    - Server: Webサーバーの情報
    - Set-Cookie: Cookieの情報
  - エンティティヘッダー
    - Allow: 利用可能なHTTPメソッドの情報
    - Content-Encoding: コンテンツのエンコード方式
    - Content-Language: コンテンツの言語
    - Content_Length: コンテンツのサイズ
    - Content_Type: コンテンツの種類
    - Expired: コンテンツの有効期限
    - Last-Modified: コンテンツの最終更新時刻

- User-Agent
  - アクセスしているブラウザのバージョンや種類を示す
  - 同じuser_AgentだとWebページアクセスできなくなることがある。
    - そのような場合、User-Agentを切り替える必要がある。
  - [What is my User Agent?](https://www.whatismybrowser.com/detect/what-is-my-user-agent)
  - [What is my ip?](https://whatismyipaddress.com/)

## HTTPコネクション

- HTTP
  - ステートレスな通信プロトコル
  - 状態を保持しない
- HTTP cookie
  - ステートフルやりとり
- HTTP KeepAlive方式
  - リクエストごとにコネクションを確立する必要がない

## HTTPとWebアプリケーション

- Webアプリケーション
  - Webブラウザ上で機能するアプリケーション
  - プレゼンテーション層
    - Webサーバー
    - 冗長化
  - アプリケーション層
    - アプリケーションサーバー
  - データ層
    - データベースサーバー
    - 冗長化
  - キャッシュサーバー
    - HTTPリクエストに対するHTTPレスポンスを記憶する

## HTTPと認証

- HTTP認証
  - ベーシック認証
  - フォームベース認証

```bash
scrapy shell "https://tabelog.com/"

from scrapy.http import FormRequest, Request

data = {
    "LstKind": "1",
    "voluntary_search": "1",
    "lid": "yoyaku-search_header",
    "sa": "渋谷駅",
    "sk": "焼肉",
    "vac_net": "1",
    "search_date": "2023/7/5(水)",
    "svd": "20230705",
    "svt": "2000",
    "svps": "10",
    "hfc": "1",
    "area_datatype": "RailroadStation",
    "area_id": "4698",
    "key_datatype": "keyword",
}

page = FormRequest('https://tabelog.com/rst/rstsearch/', formdata = data)

fetch(page)
view(response)
```

## HTTP Cookie

- requests.Session()

```bash
curl --verbose -H 'Cookie: name=scrapy; ver=111' http://example.com
*   Trying [2606:2800:220:1:248:1893:25c8:1946]:80...
* Connected to example.com (2606:2800:220:1:248:1893:25c8:1946) port 80 (#0)
> GET / HTTP/1.1
> Host: example.com
> User-Agent: curl/7.88.1
> Accept: */*
> Cookie: name=scrapy; ver=111
> 
< HTTP/1.1 200 OK
< Accept-Ranges: bytes
< Age: 340560
< Cache-Control: max-age=604800
< Content-Type: text/html; charset=UTF-8
< Date: Tue, 04 Jul 2023 10:34:49 GMT
< Etag: "3147526947"
< Expires: Tue, 11 Jul 2023 10:34:49 GMT
< Last-Modified: Thu, 17 Oct 2019 07:18:26 GMT
< Server: ECS (sab/5695)
< Vary: Accept-Encoding
< X-Cache: HIT
< Content-Length: 1256
< 
<!doctype html>
<html>
<head>
    <title>Example Domain</title>

    <meta charset="utf-8" />
    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <style type="text/css">
    body {
        background-color: #f0f0f2;
        margin: 0;
        padding: 0;
        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;
        
    }
    div {
        width: 600px;
        margin: 5em auto;
        padding: 2em;
        background-color: #fdfdff;
        border-radius: 0.5em;
        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);
    }
    a:link, a:visited {
        color: #38488f;
        text-decoration: none;
    }
    @media (max-width: 700px) {
        div {
            margin: 0 auto;
            width: auto;
        }
    }
    </style>    
</head>

<body>
<div>
    <h1>Example Domain</h1>
    <p>This domain is for use in illustrative examples in documents. You may use this
    domain in literature without prior coordination or asking for permission.</p>
    <p><a href="https://www.iana.org/domains/example">More information...</a></p>
</div>
</body>
</html>
* Connection #0 to host example.com left intact
```

In [1]:
import requests

url = "http://www.webscrapingfordatascience.com/cookielogin/secret.php"
r = requests.get(url)

print(r.text)
# うーん...ログインしていないようですね。

Hmm... it seems you are not logged in


In [2]:
import requests

# Login
url = "http://www.webscrapingfordatascience.com/cookielogin/"
data_post = {"username": "testuser", "password": "password"}
r = requests.post(url, data = data_post)

# Cookieを取得して、secretページにアクセス
my_cookie = r.cookies
url = "http://www.webscrapingfordatascience.com/cookielogin/secret.php"
r = requests.get(url, cookies=my_cookie)

print(r.text)
# これは秘密のコードです： 1234

This is a secret code: 1234


In [3]:
import requests

# Login
url = "http://www.webscrapingfordatascience.com/redirlogin/"
data_post = {"username": "testuser", "password": "password"}
# リダイレクトされない設定
r = requests.post(url, data = data_post, allow_redirects=False)

# Cookieを取得して、secretページにアクセス
my_cookie = r.cookies
url = "http://www.webscrapingfordatascience.com/redirlogin/secret.php"
r = requests.get(url, cookies = my_cookie)

print(r.text)

This is a secret code: 1234


In [4]:
# ログインページにアクセスしたかどうか、ログイン後にCookieのセッションIDを変更するようなサイトの場合

import requests

# Formを取得
url = "http://www.webscrapingfordatascience.com/trickylogin/"
r = requests.post(url)
my_cookie = r.cookies
print(my_cookie)

# 先程のCookieを使ってpostを実行
r = requests.post(
    url,
    params={"p": "login"},
    data = {"username": "testuser", "password": "password"},
    allow_redirects = False,
    cookies = my_cookie
)

# Cookieを更新する
my_cookie = r.cookies
print(my_cookie)

# 秘密のページへアクセス
r = requests.get(url, params={"p": "protected"}, cookies=my_cookie)
print(r.text)
# <RequestsCookieJar[<Cookie PHPSESSID=mjq8t3p915f99o14k9sambtgod for www.webscrapingfordatascience.com/>]>
# <RequestsCookieJar[<Cookie PHPSESSID=t3vc5ij23ft7kl8b82itgs71oe for www.webscrapingfordatascience.com/>]>
# これがあなたの秘密のコードです： 3838.

<RequestsCookieJar[<Cookie PHPSESSID=mjq8t3p915f99o14k9sambtgod for www.webscrapingfordatascience.com/>]>
<RequestsCookieJar[<Cookie PHPSESSID=t3vc5ij23ft7kl8b82itgs71oe for www.webscrapingfordatascience.com/>]>
Here is your secret code: 3838.


<https://www.youtube.com/watch?v=NvkWfjMWn7Y>

[REquestBin.com](https://requestbin.com/r/enldafevk2rhb/)

In [26]:
import requests

# custom headers
headers = {
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "accept-encoding": "gzip, deflate, br",
    "accept-language": "ja,en-US;q=0.9,en;q=0.8,zh-CN;q=0.7,zh;q=0.6",
    "cache-control": "max-age=0",
    "cookie": "_gcl_au=1.1.2053070165.1592979574; _ga=GA1.2.1180196223.1592979574; _gid=GA1.2.1504925251.1592979574; _lr_uf_-dhjtrz=a5956b98-e3b3-41d8-ab3a-3f9c182bd391; _hjid=ec0ef6a8-a0b1-431d-abe1-01397ae26645; ki_r=; _uetsid=11adf75b-e96c-d7d7-9ea2-dfa133728451; _uetvid=d6cd5e1d-cdde-40f2-01b6-f60ff4361f57; _gat_UA-128559955-1=1; _lr_tabs_-dhjtrz%2Fpd={%22sessionID%22:0%2C%22recordingID%22:%224-be179f28-1ed2-4d5d-81ff-e6d0d409ac9a%22%2C%22lastActivity%22:1592986151170}; _lr_hb_-dhjtrz%2Fpd={%22heartbeat%22:1592986151172}; ki_t=1592979575629%3B1592979575629%3B1592986152027%3B1%3B5; amplitude_id_eadd7e2135597c308ef5d9db3651c843requestbin.com=eyJkZXZpY2VJZCI6IjdjYTczN2U3LTkyMzMtNDllOC1iOGUxLTEwMGRiZGM1YWEwM1IiLCJ1c2VySWQiOm51bGwsIm9wdE91dCI6ZmFsc2UsInNlc3Npb25JZCI6MTU5Mjk4NTczNzcyNiwibGFzdEV2ZW50VGltZSI6MTU5Mjk4NjE1MjEyNywiZXZlbnRJZCI6NiwiaWRlbnRpZnlJZCI6MCwic2VxdWVuY2VOdW1iZXIiOjZ9",
    "if-modified-since": "Wed, 01 Apr 2020 23:15:20 GMT",
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "none",
    "sec-fetch-user": "?1",
    "upgrade-insecure-requests": "1",
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36"
}

# requestbin.com URL
url = 'https://enldafevk2rhb.x.pipedream.net/'

# make HTTP GET request to requestbin.com
response = requests.get(url, headers=headers)
print(response.text)

2023-07-04 21:30:46 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): enldafevk2rhb.x.pipedream.net:443


2023-07-04 21:30:46 [urllib3.connectionpool] DEBUG: https://enldafevk2rhb.x.pipedream.net:443 "GET / HTTP/1.1" 200 16


{"success":true}


```python
# cookie_1.py
import scrapy
from scrapy.crawler import CrawlerProcess

# spider class
class HeadersCookies(scrapy.Spider):
    name = "headerscookies"

    url = "https://enldafevk2rhb.x.pipedream.net/"

    # custom headers
    headers = {
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "accept-encoding": "gzip, deflate, br",
        "accept-language": "ja,en-US;q=0.9,en;q=0.8,zh-CN;q=0.7,zh;q=0.6",
        "cache-control": "max-age=0",
        "cookie": "_gcl_au=1.1.2053070165.1592979574; _ga=GA1.2.1180196223.1592979574; _gid=GA1.2.1504925251.1592979574; _lr_uf_-dhjtrz=a5956b98-e3b3-41d8-ab3a-3f9c182bd391; _hjid=ec0ef6a8-a0b1-431d-abe1-01397ae26645; ki_r=; _uetsid=11adf75b-e96c-d7d7-9ea2-dfa133728451; _uetvid=d6cd5e1d-cdde-40f2-01b6-f60ff4361f57; _gat_UA-128559955-1=1; _lr_tabs_-dhjtrz%2Fpd={%22sessionID%22:0%2C%22recordingID%22:%224-be179f28-1ed2-4d5d-81ff-e6d0d409ac9a%22%2C%22lastActivity%22:1592986151170}; _lr_hb_-dhjtrz%2Fpd={%22heartbeat%22:1592986151172}; ki_t=1592979575629%3B1592979575629%3B1592986152027%3B1%3B5; amplitude_id_eadd7e2135597c308ef5d9db3651c843requestbin.com=eyJkZXZpY2VJZCI6IjdjYTczN2U3LTkyMzMtNDllOC1iOGUxLTEwMGRiZGM1YWEwM1IiLCJ1c2VySWQiOm51bGwsIm9wdE91dCI6ZmFsc2UsInNlc3Npb25JZCI6MTU5Mjk4NTczNzcyNiwibGFzdEV2ZW50VGltZSI6MTU5Mjk4NjE1MjEyNywiZXZlbnRJZCI6NiwiaWRlbnRpZnlJZCI6MCwic2VxdWVuY2VOdW1iZXIiOjZ9",
        "if-modified-since": "Wed, 01 Apr 2020 23:15:20 GMT",
        "sec-fetch-dest": "document",
        "sec-fetch-mode": "navigate",
        "sec-fetch-site": "none",
        "sec-fetch-user": "?1",
        "upgrade-insecure-requests": "1",
        "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36"
    }

    def start_requests(self):
        # make HTTP GET request to requestbin.com
        yield scrapy.Request(
            url=self.url,
            headers=self.headers,
            callback=self.parse
        )

    def parse(self, response):
        print(response.text)


if __name__ == "__main__":
    process = CrawlerProcess()
    process.crawl(HeadersCookies)
    process.start()

```

```bash
python ./scrapy_note/ch_08/cookie_1.py
2023-07-04 22:08:53 [scrapy.utils.log] INFO: Scrapy 2.9.0 started (bot: scrapybot)
2023-07-04 22:08:53 [scrapy.utils.log] INFO: Versions: lxml 4.9.2.0, libxml2 2.9.14, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.1, Twisted 22.10.0, Python 3.10.11 (main, Apr 24 2023, 17:34:58) [Clang 14.0.3 (clang-1403.0.22.14.1)], pyOpenSSL 23.2.0 (OpenSSL 3.1.1 30 May 2023), cryptography 41.0.1, Platform macOS-13.4-x86_64-i386-64bit
2023-07-04 22:08:53 [scrapy.crawler] INFO: Overridden settings:
{}
2023-07-04 22:08:53 [py.warnings] WARNING: /Users/takeru/@LEARNING/Python/scrapy_docs/venv/lib/python3.10/site-packages/scrapy/utils/request.py:232: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting for information on how to handle this deprecation.
  return cls(crawler)

2023-07-04 22:08:53 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2023-07-04 22:08:53 [scrapy.extensions.telnet] INFO: Telnet Password: 9847fc96658dcc2f
2023-07-04 22:08:53 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2023-07-04 22:08:53 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'scrapy.downloadermiddlewares.redirect.RedirectMiddleware',
 'scrapy.downloadermiddlewares.cookies.CookiesMiddleware',
 'scrapy.downloadermiddlewares.httpproxy.HttpProxyMiddleware',
 'scrapy.downloadermiddlewares.stats.DownloaderStats']
2023-07-04 22:08:53 [scrapy.middleware] INFO: Enabled spider middlewares:
['scrapy.spidermiddlewares.httperror.HttpErrorMiddleware',
 'scrapy.spidermiddlewares.offsite.OffsiteMiddleware',
 'scrapy.spidermiddlewares.referer.RefererMiddleware',
 'scrapy.spidermiddlewares.urllength.UrlLengthMiddleware',
 'scrapy.spidermiddlewares.depth.DepthMiddleware']
2023-07-04 22:08:53 [scrapy.middleware] INFO: Enabled item pipelines:
[]
2023-07-04 22:08:53 [scrapy.core.engine] INFO: Spider opened
2023-07-04 22:08:53 [scrapy.extensions.logstats] INFO: Crawled 0 pages (at 0 pages/min), scraped 0 items (at 0 items/min)
2023-07-04 22:08:53 [scrapy.extensions.telnet] INFO: Telnet console listening on 127.0.0.1:6047
2023-07-04 22:08:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://enldafevk2rhb.x.pipedream.net/> (referer: None)
{"success":true}
2023-07-04 22:08:54 [scrapy.core.engine] INFO: Closing spider (finished)
2023-07-04 22:08:54 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 623,
 'downloader/request_count': 1,
 'downloader/request_method_count/GET': 1,
 'downloader/response_bytes': 224,
 'downloader/response_count': 1,
 'downloader/response_status_count/200': 1,
 'elapsed_time_seconds': 1.018926,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2023, 7, 4, 13, 8, 54, 627782),
 'log_count/DEBUG': 2,
 'log_count/INFO': 10,
 'log_count/WARNING': 1,
 'memusage/max': 50376704,
 'memusage/startup': 50376704,
 'response_received_count': 1,
 'scheduler/dequeued': 1,
 'scheduler/dequeued/memory': 1,
 'scheduler/enqueued': 1,
 'scheduler/enqueued/memory': 1,
 'start_time': datetime.datetime(2023, 7, 4, 13, 8, 53, 608856)}
2023-07-04 22:08:54 [scrapy.core.engine] INFO: Spider closed (finished)
```

```python
import scrapy
from scrapy.crawler import CrawlerProcess

# spider class
class HeadersCookies(scrapy.Spider):
    name = "headerscookies"

    url = "https://enldafevk2rhb.x.pipedream.net/"

    # custom headers
    headers = {
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "accept-encoding": "gzip, deflate, br",
        "accept-language": "ja,en-US;q=0.9,en;q=0.8,zh-CN;q=0.7,zh;q=0.6",
        "cache-control": "max-age=0",
        "if-modified-since": "Wed, 01 Apr 2020 23:15:20 GMT",
        "sec-fetch-dest": "document",
        "sec-fetch-mode": "navigate",
        "sec-fetch-site": "none",
        "sec-fetch-user": "?1",
        "upgrade-insecure-requests": "1",
        "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36"
    }

    raw_cookie = "_gcl_au=1.1.2053070165.1592979574; _ga=GA1.2.1180196223.1592979574; _gid=GA1.2.1504925251.1592979574; _lr_uf_-dhjtrz=a5956b98-e3b3-41d8-ab3a-3f9c182bd391; _hjid=ec0ef6a8-a0b1-431d-abe1-01397ae26645; ki_r=; _uetsid=11adf75b-e96c-d7d7-9ea2-dfa133728451; _uetvid=d6cd5e1d-cdde-40f2-01b6-f60ff4361f57; _gat_UA-128559955-1=1; _lr_tabs_-dhjtrz%2Fpd={%22sessionID%22:0%2C%22recordingID%22:%224-be179f28-1ed2-4d5d-81ff-e6d0d409ac9a%22%2C%22lastActivity%22:1592986151170}; _lr_hb_-dhjtrz%2Fpd={%22heartbeat%22:1592986151172}; ki_t=1592979575629%3B1592979575629%3B1592986152027%3B1%3B5; amplitude_id_eadd7e2135597c308ef5d9db3651c843requestbin.com=eyJkZXZpY2VJZCI6IjdjYTczN2U3LTkyMzMtNDllOC1iOGUxLTEwMGRiZGM1YWEwM1IiLCJ1c2VySWQiOm51bGwsIm9wdE91dCI6ZmFsc2UsInNlc3Npb25JZCI6MTU5Mjk4NTczNzcyNiwibGFzdEV2ZW50VGltZSI6MTU5Mjk4NjE1MjEyNywiZXZlbnRJZCI6NiwiaWRlbnRpZnlJZCI6MCwic2VxdWVuY2VOdW1iZXIiOjZ9"

    # parse cookies
    def parse_cookies(self, raw_cookies):
        cookies = {}
        for cookie in raw_cookies.split('; '):
            try:
                key = cookie.split('=')[0]
                val = cookie.split('=')[1]
                cookies[key] = val
            except:
                pass

        return cookies

    def start_requests(self):
        # make HTTP GET request to requestbin.com
        yield scrapy.Request(
            url = self.url,
            headers = self.headers,
            cookies = self.parse_cookies(self.raw_cookie),
            callback = self.parse
        )

    def parse(self, response):
        print(response.text)


if __name__ == "__main__":
    process = CrawlerProcess()
    process.crawl(HeadersCookies)
    process.start()
```

```bash
python ./scrapy_note/ch_08/cookie_2.py
2023-07-04 22:10:24 [scrapy.utils.log] INFO: Scrapy 2.9.0 started (bot: scrapybot)
2023-07-04 22:10:24 [scrapy.utils.log] INFO: Versions: lxml 4.9.2.0, libxml2 2.9.14, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.1, Twisted 22.10.0, Python 3.10.11 (main, Apr 24 2023, 17:34:58) [Clang 14.0.3 (clang-1403.0.22.14.1)], pyOpenSSL 23.2.0 (OpenSSL 3.1.1 30 May 2023), cryptography 41.0.1, Platform macOS-13.4-x86_64-i386-64bit
2023-07-04 22:10:24 [scrapy.crawler] INFO: Overridden settings:
{}
2023-07-04 22:10:24 [py.warnings] WARNING: /Users/takeru/@LEARNING/Python/scrapy_docs/venv/lib/python3.10/site-packages/scrapy/utils/request.py:232: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting for information on how to handle this deprecation.
  return cls(crawler)

2023-07-04 22:10:24 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2023-07-04 22:10:24 [scrapy.extensions.telnet] INFO: Telnet Password: 4b650b092bc50ef6
2023-07-04 22:10:24 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2023-07-04 22:10:24 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'scrapy.downloadermiddlewares.redirect.RedirectMiddleware',
 'scrapy.downloadermiddlewares.cookies.CookiesMiddleware',
 'scrapy.downloadermiddlewares.httpproxy.HttpProxyMiddleware',
 'scrapy.downloadermiddlewares.stats.DownloaderStats']
2023-07-04 22:10:24 [scrapy.middleware] INFO: Enabled spider middlewares:
['scrapy.spidermiddlewares.httperror.HttpErrorMiddleware',
 'scrapy.spidermiddlewares.offsite.OffsiteMiddleware',
 'scrapy.spidermiddlewares.referer.RefererMiddleware',
 'scrapy.spidermiddlewares.urllength.UrlLengthMiddleware',
 'scrapy.spidermiddlewares.depth.DepthMiddleware']
2023-07-04 22:10:24 [scrapy.middleware] INFO: Enabled item pipelines:
[]
2023-07-04 22:10:24 [scrapy.core.engine] INFO: Spider opened
2023-07-04 22:10:25 [scrapy.extensions.logstats] INFO: Crawled 0 pages (at 0 pages/min), scraped 0 items (at 0 items/min)
2023-07-04 22:10:25 [scrapy.extensions.telnet] INFO: Telnet console listening on 127.0.0.1:6047
2023-07-04 22:10:26 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://enldafevk2rhb.x.pipedream.net/> (referer: None)
{"success":true}
2023-07-04 22:10:26 [scrapy.core.engine] INFO: Closing spider (finished)
2023-07-04 22:10:26 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 1514,
 'downloader/request_count': 1,
 'downloader/request_method_count/GET': 1,
 'downloader/response_bytes': 224,
 'downloader/response_count': 1,
 'downloader/response_status_count/200': 1,
 'elapsed_time_seconds': 1.556411,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2023, 7, 4, 13, 10, 26, 594415),
 'log_count/DEBUG': 2,
 'log_count/INFO': 10,
 'log_count/WARNING': 1,
 'memusage/max': 50520064,
 'memusage/startup': 50520064,
 'response_received_count': 1,
 'scheduler/dequeued': 1,
 'scheduler/dequeued/memory': 1,
 'scheduler/enqueued': 1,
 'scheduler/enqueued/memory': 1,
 'start_time': datetime.datetime(2023, 7, 4, 13, 10, 25, 38004)}
2023-07-04 22:10:26 [scrapy.core.engine] INFO: Spider closed (finished)
```